In [1]:
import numpy as np 
import re 
from bs4 import BeautifulSoup
import spacy 
import sys
sys.path.append('scripts')
import pysbd
import json 
import re 
import random 
from spacy.tokens import DocBin
import jsonlines
from spacy.tokens import Span
from prodigy.components.db import connect
from prettytable import PrettyTable
import srsly
from prodigy.components.db import connect
from pathlib import Path 
from spacy import displacy
from pysbd.utils import PySBDFactory
from components import *
from string import punctuation
from spacy.tokens import Doc
from collections import Counter

from configparser import ConfigParser, ExtendedInterpolation
from collections import defaultdict
from itertools import combinations, product
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from gensim.matutils import kullback_leibler, hellinger
from gensim.models import ldamodel
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, Similarity
from gensim.utils import simple_preprocess
from IPython.core.display import display, HTML
from IPython.display import Image
from IPython.lib.display import YouTubeVideo
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from spacy.matcher import Matcher
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
import spacy

# increase display of columns in pandas
pd.set_option('display.max_colwidth', 200)
#Visualization des phrases et des ents 

# sentence_spans = list(doc.sents)
# displacy.render(sentence_spans, style="ent")

In [2]:

nlp = spacy.load("/Users/anis/test_labnbook/math_ner/correct_ner/packages/fr_LabnbookNer-0.0.0/fr_LabnbookNer/fr_LabnbookNer-0.0.0")

text = "Expliqudfdfez comment vous déterminez les concentrations des solutions de la gamme étalon. Tout d'abord, on va déterminer les concentrations maximale et minimale de notre gamme étalon. Pour cela, on sait que notre spectrophotomètre est linéaire pour des absorbances se trouvant en 0.05 et 2. Donc grace à la formule A=C*E*l on peut déterminer les concentrations des extrêmes : § C min=A E×l=0.05104×1=5×10-6mol/L § C max=2104×1=2×10-4mol/L § Donc notre gamme étalon doit être entre 5*10-6 et 2*10-4mol/L. Ensuite, on peut maintenant déterminer les volumes maximum et minimum à prélever de solution de sel de Mohr grâce à la formule Cm*Vm=Cf*Vf § V min=Cf×Vf Cm=5×10-6×0.01 2×10-3=2.5×10-5 L=25µL § V max=2×10-4×0.01 2×10-3=1×10-3 L=1mL=1000µL § Donc on va devoir prélever des volumes de sel de Mohr entre 25 et 1000µL.Enfin, nous avons 8 tubes plastiques de 10mL donc on va faire 8solutions pour notre gamme étalon et nous voulons des valeurs à peu près espacées de la même valeurs pour avoir une courbe facile a determiner. On va donc prendre les valeurs de volumes de solution de Mohr suivantes (en µL): 25, 100, 250, 400, 550, 700, 850, 1000. Ce qui nous donne des concentrations qui leur correspondent que l'on obtient grâce a la formule Cm*Vm=Cf*Vf : § ¥¥Volume de Sel de Mohr à prélever  (en µL)¥ Concentration des solutions de la gamme étalon (en mol/L¥ 25¥ 5*10-6¥ 100¥ 2*10-5¥ 250¥ 5*10-5¥  400¥ 8*10-5¥ 550¥ 1.1*10-4¥ 700¥ 1.4*10-4¥ 850¥ 1.7*10-4¥ 1000¥ 2*10-4¥ ¥¥ "

displacy.render(list(nlp(text).sents), style="ent")


In [4]:
#! streamlit run /Users/anis/test_labnbook/math_ner/train_ner/scripts/visualize_html.py 380148     

In [3]:
nlp.pipe_names

['tok2vec',
 'Find_LatexTable',
 'pysbd_sentencizer',
 'ner',
 'custom_sentencizer',
 'entity_retokenizer']

In [4]:
text = "pour un débit de 500 L/h on obtient hA=305mm et hB=20kg on a delta h=190mm §"

displacy.render(list(nlp(text).sents), style="ent")


In [5]:

doc1 = nlp("l'exemple de d'embarquepment et s'amuser")

for token in doc1:
    print(token.text, token.has_vector,token.vector_norm, token.is_oov,"\n")

l'exemple True 22.953375 True 

de True 23.157715 True 

d'embarquepment True 23.732521 True 

et True 22.233355 True 

s'amuser True 21.398802 True 



In [17]:
nlp = spacy.load("fr_core_news_md")

doc1 = nlp("bonjour salut coucou rgrgrgrg ")

for token in doc1:
    print(token.text, token.has_vector,token.vector_norm, token.is_oov,"\n")

bonjour True 20.88204 False 

salut True 25.95254 False 

coucou True 20.718126 False 

rgrgrgrg False 0.0 True 



In [20]:
vocab  = nlp.vocab

In [211]:
nlp.vocab

300

In [5]:
nlp2 = spacy.load("training/model-best")


In [1]:
from sense2vec import Sense2Vec


In [6]:
nlp2.add_pipe("sense2vec")

AttributeError: 'spacy.vocab.Vocab' object has no attribute 'vectors_ngrams'

In [23]:
doc1 = nlp2("bonjour Bonjour salut coucou rgrgrgrg ")

for token in doc1:
    print(token.text, token.has_vector,token.vector_norm, token.is_oov,"\n")

bonjour True 21.901 True 

Bonjour True 21.74684 True 

salut True 21.572641 True 

coucou True 21.452328 True 

rgrgrgrg True 17.624662 True 



In [78]:
import difflib
text1 = "bonjour $$ latex + alpha $$"
text2 = "bonjour $$ latex + beta $$"

doc1 = nlp2(text1)
doc2 = nlp2(text2)

s1 = [token.text for token in doc1 if not token.ent_type_]
s2 = [token.text for token in doc2 if not token.ent_type_]

matcher = difflib.SequenceMatcher(a=s1, b=s2)
print("Matching Sequences:")
for match in matcher.get_matching_blocks():
    print("Match             : {}".format(match))
    print("Matching Sequence : {}".format(s1[match.a:match.a+match.size]))
    #print the ratio



Matching Sequences:
Match             : Match(a=0, b=0, size=1)
Matching Sequence : ['bonjour']
Match             : Match(a=1, b=1, size=0)
Matching Sequence : []
Match             : ('equal', 0, 1, 0, 1)
Matching Sequence : ['bonjour']


In [79]:
for match in matcher.get_opcodes():
    print("Match             : {}".format(match))
    print("Matching Sequence : {}".format(s1[match[1]:match[2]]))
    #print the ratio

Match             : ('equal', 0, 1, 0, 1)
Matching Sequence : ['bonjour']


In [86]:
# s1 = [token.text for token in doc1 if  token.ent_type_]
s1 = [ent for ent in doc1.ents ]
# s2 = [token.text for token in doc2 if  token.ent_type_]
s2 = [ent for ent in doc2.ents ]

matcher = difflib.SequenceMatcher(a=s1, b=s2)
print("Matching Sequences:")
for match in matcher.get_matching_blocks():
    print("Match             : {}".format(match))
    print("Matching Sequence : {}".format(s1[match.a:match.a+match.size]))

Matching Sequences:
Match             : Match(a=1, b=1, size=0)
Matching Sequence : []


In [87]:
for match in matcher.get_opcodes():
    print("Match             : {}".format(match))
    print("Matching Sequence : {}".format(s1[match[1]:match[2]]))
    #print the ratio

Match             : ('replace', 0, 1, 0, 1)
Matching Sequence : [$$ latex + alpha $$]


In [90]:
for tag, i1, i2, j1, j2 in matcher.get_opcodes():
   print(tag,'\n',s1[i1:i2], s2[j1:j2])
   #print ratio of tag

replace 
 [$$ latex + alpha $$] [$$ latex + beta $$]


In [97]:
s1 = "It is warm. It is summer.".split(' ')
s2 = "It is not warm. It is winter.".split(' ')
matcher = difflib.SequenceMatcher(a=s1, b=s2)
print("Matching Sequences:")
for match in matcher.get_matching_blocks():
    print("Match             : {}".format(match))
    print("Matching Sequence : {}".format(s1[match.a:match.a+match.size]))


Matching Sequences:
Match             : Match(a=0, b=0, size=2)
Matching Sequence : ['It', 'is']
['It', 'is']
Match             : Match(a=2, b=3, size=3)
Matching Sequence : ['warm.', 'It', 'is']
['warm.', 'It', 'is']
Match             : Match(a=6, b=7, size=0)
Matching Sequence : []
[]


In [136]:
import re

def tokenize(s):
    return re.split('\s+', s)
def untokenize(ts):
    return ' '.join(ts)
        
def equalize(s1, s2):
    l1 = tokenize(s1)
    l2 = tokenize(s2)
    res1 = []
    res2 = []
    prev = difflib.Match(0,0,0)
    for match in difflib.SequenceMatcher(a=l1, b=l2).get_matching_blocks():
        if (prev.a + prev.size != match.a):
            for i in range(prev.a + prev.size, match.a):
                res2 += ['_' * len(l1[i])]
            res1 += l1[prev.a + prev.size:match.a]
        if (prev.b + prev.size != match.b):
            for i in range(prev.b + prev.size, match.b):
                res1 += ['_' * len(l2[i])]
            res2 += l2[prev.b + prev.size:match.b]
        res1 += l1[match.a:match.a+match.size]
        res2 += l2[match.b:match.b+match.size]
        prev = match
    return untokenize(res1), untokenize(res2)

In [137]:
equalize("It is warm. It is summer.", "It is warm. It is good. My name is Anis.")

('It is warm. It is summer. _____ __ ____ __ _____',
 'It is warm. It is _______ good. My name is Anis.')

In [105]:
def insert_newlines(string, every=40, window=10):
    result = []
    from_str = string
    while len(from_str) > 0:
        cut_off = every
        if len(from_str) > every:
            while (from_str[cut_off - 1] != ' ') and (cut_off > (every-window)):
                cut_off -= 1
        else:
            cut_off = len(from_str)
        part = from_str[:cut_off]
        result += [part]
        from_str = from_str[cut_off:]
    return result

In [106]:
def show_comparison(s1, s2, width=40, margin=10, sidebyside=True, compact=False):
    s1, s2 = equalize(s1,s2)

    if sidebyside:
        s1 = insert_newlines(s1, width, margin)
        s2 = insert_newlines(s2, width, margin)
        if compact:
            for i in range(0, len(s1)):
                lft = re.sub(' +', ' ', s1[i].replace('_', '')).ljust(width)
                rgt = re.sub(' +', ' ', s2[i].replace('_', '')).ljust(width) 
                print(lft + ' | ' + rgt + ' | ')        
        else:
            for i in range(0, len(s1)):
                lft = s1[i].ljust(width)
                rgt = s2[i].ljust(width)
                print(lft + ' | ' + rgt + ' | ')
    else:
        print(s1)
        print(s2)

In [109]:
x = []
repr(x.append(0),2)

AttributeError: 'list' object has no attribute 'appen'

In [112]:
# repeat x.appen 5 times 
x = []
for i in range(0,5):
    x.append(0)

print(x)

[0, 0, 0, 0, 0]


# New

In [2]:
nlp = spacy.load("/Users/anis/test_labnbook/math_ner/correct_ner/packages/fr_LabnbookNer-0.0.0/fr_LabnbookNer/fr_LabnbookNer-0.0.0")
# displacy.render(list(nlp(text).sents), style="ent")

In [3]:
text = " ceci est une formule mathématique  ¥¥200mm¥¥  et celle-ci est une autre formule mathématique $$ \\frac{1}{2} $$ " 

In [4]:
displacy.render(list(nlp(text).sents), style="ent")